# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df.describe(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


(                  id
 count    8188.000000
 mean   564699.749878
 std    219085.845672
 min    182988.000000
 25%    373637.500000
 50%    566840.500000
 75%    758078.000000
 max    937217.000000,
 None)

In [7]:
df['id'].describe()

count      8188.000000
mean     564699.749878
std      219085.845672
min      182988.000000
25%      373637.500000
50%      566840.500000
75%      758078.000000
max      937217.000000
Name: id, dtype: float64

In [10]:
# counting viewers and clickers
viewers = df[df['action'] == 'view']['id'].nunique()
clickers = df[df['action'] == 'click']['id'].nunique()

print("Number of viewers:", viewers)
print("Number of clickers:", clickers)


Number of viewers: 6328
Number of clickers: 1860


In [11]:
# Analyzing anommallies
anomalies = df[(df['action'] == 'click') & (df['action'] != 'view')]
print("Anomalies with data:")
print(anomalies)


Anomalies with data:
                       timestamp      id       group action
8     2016-09-24 20:58:01.948663  349125  experiment  click
12    2016-09-24 21:06:27.553057  601714  experiment  click
15    2016-09-24 21:30:02.739756  487634  experiment  click
17    2016-09-24 23:01:12.108316  468601  experiment  click
21    2016-09-25 00:01:47.933853  555973  experiment  click
...                          ...     ...         ...    ...
8162  2017-01-17 23:20:35.483601  451198     control  click
8164  2017-01-17 23:47:58.209653  252195     control  click
8167  2017-01-18 00:56:24.554729  344770  experiment  click
8180  2017-01-18 08:54:56.879682  615849  experiment  click
8182  2017-01-18 09:09:17.363917  795585     control  click

[1860 rows x 4 columns]


In [12]:
# Checking overlap
overlap = df.groupby(['id']).nunique()['group'].value_counts()
print("Overlap between Control and Experiment Groups:")
print(overlap)


Overlap between Control and Experiment Groups:
group
1    6328
Name: count, dtype: int64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [13]:
#Your code here
from scipy.stats import chi2_contingency

# Filter out anomalies where click action occurs without view action
valid_data = df[df['action'] != 'click'] 

# Create a contingency table of observed frequencies
contingency_table = pd.crosstab(valid_data['group'], valid_data['action'])

# Perform the chi-square test
chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)


Chi-square statistic: 0.0
p-value: 1.0


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [15]:
# Filter out anomalies where click action occurs without view action
valid_data = df[df['action'] != 'click']

# Create a contingency table of observed frequencies
contingency_table = pd.crosstab(valid_data['group'], valid_data['action'])

# Calculate click-through rate for the control group
control_click_through_rate = contingency_table.loc['control', 'view'] / contingency_table.loc['control'].sum()

# Calculate the expected number of clicks for the experiment group
experiment_expected_clicks = control_click_through_rate * contingency_table.loc['experiment'].sum()

print("Expected number of clicks for the experiment group:", experiment_expected_clicks)


Expected number of clicks for the experiment group: 2996.0


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [16]:
#Your code here
import math

actual_clicks = contingency_table.loc['experiment', 'view']
z = (actual_clicks - experiment_expected_clicks) / math.sqrt(experiment_expected_clicks)

print("Number of standard deviations:", z)


Number of standard deviations: 0.0


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [18]:
#Your code here
# Calculate the p-value
p_value = 1 - stats.norm.cdf(z)

print("p-value:", p_value)


p-value: 0.5


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Display the first few rows of the dataframe
df.head()

# Perform exploratory analysis
df.describe()
df.info()

# Count viewers and clickers
viewers = df[df['action'] == 'view']['id'].nunique()
clickers = df[df['action'] == 'click']['id'].nunique()

print("Number of viewers:", viewers)
print("Number of clickers:", clickers)

# Analyze anomalies
anomalies = df[(df['action'] == 'click') & (df['action'] != 'view')]
print("Anomalies with data:")
print(anomalies)

# Check overlap between control and experiment groups
overlap = df.groupby(['id']).nunique()['group'].value_counts()
print("Overlap between Control and Experiment Groups:")
print(overlap)

# Perform a statistical test
from scipy.stats import chi2_contingency

# Filter out anomalies where click action occurs without view action
valid_data = df[df['action'] != 'click']

# Create a contingency table of observed frequencies
contingency_table = pd.crosstab(valid_data['group'], valid_data['action'])

# Perform the chi-square test
chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)

# Calculate the expected number of clicks for the experiment group
control_click_through_rate = contingency_table.loc['control', 'view'] / contingency_table.loc['control'].sum()
experiment_expected_clicks = control_click_through_rate * contingency_table.loc['experiment'].sum()

print("Expected number of clicks for the experiment group:", experiment_expected_clicks)

# Calculate the number of standard deviations
actual_clicks = contingency_table.loc['experiment', 'view']
z = (actual_clicks - experiment_expected_clicks) / np.sqrt(experiment_expected_clicks)

print("Number of standard deviations:", z)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
Number of viewers: 6328
Number of clickers: 1860
Anomalies with data:
                       timestamp      id       group action
8     2016-09-24 20:58:01.948663  349125  experiment  click
12    2016-09-24 21:06:27.553057  601714  experiment  click
15    2016-09-24 21:30:02.739756  487634  experiment  click
17    2016-09-24 23:01:12.108316  468601  experiment  click
21    2016-09-25 00:01:47.933853  555973  experiment  click
...                          ...     ...         ...    ...
8162  2017-01-17 23:20:35.483601  451198     control  click
8164  2017-01-17 23:47:58.209653  252195     control  clic